<a href="https://colab.research.google.com/github/sAI-2025/AI-assisted-Plant-Disease-Detection-and-Management-System-for-Sustainable-Agriculture/blob/main/Pre_trained_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!git clone https://github.com/ultralytics/yolov5.git

In [ ]:
import torch
import os

# Define paths
pt_file_path = '/content/yolov5s.pt'
export_script_path = '/content/yolov5/export.py'
output_weights_path = '/content/Weights'

# Load the pretrained PyTorch model
model = torch.load(pt_file_path, map_location=torch.device('cpu'))

# Save the model in a format suitable for export
torch.save(model.state_dict(), 'temp.pt')

# Run the export script
os.system(f'python {export_script_path} --weights temp.pt --img 640 --batch 1')

# Rename the exported weights file to .weights
os.rename('runs/hub/exp/weights/best.pt', output_weights_path)

# Remove the temporary .pt file
os.remove('temp.pt')

print(f"Converted .pt file to .weights: {output_weights_path}")


In [ ]:
pip install yolov5

In [ ]:
import cv2
import yolov5
import torch

class_names = [
    "Person", "Bicycle", "Car", "Motorcycle", "Airplane", "Bus", "Train", "Truck", "Boat",
    "Traffic light", "Fire hydrant", "Stop sign", "Parking meter", "Bench", "Bird", "Cat",
    "Dog", "Horse", "Sheep", "Cow", "Elephant", "Bear", "Zebra", "Giraffe", "Backpack",
    "Umbrella", "Handbag", "Tie", "Suitcase", "Frisbee", "Skis", "Snowboard", "Sports ball",
    "Kite", "Baseball bat", "Baseball glove", "Skateboard", "Surfboard", "Tennis racket",
    "Bottle", "Wine glass", "Cup", "Fork", "Knife", "Spoon", "Bowl", "Banana", "Apple",
    "Sandwich", "Orange", "Broccoli", "Carrot", "Hot dog", "Pizza", "Donut", "Cake", "Chair",
    "Couch", "Potted plant", "Bed", "Dining table", "Toilet", "TV", "Laptop", "Mouse",
    "Remote", "Keyboard", "Cell phone", "Microwave", "Oven", "Toaster", "Sink", "Refrigerator",
    "Book", "Clock", "Vase", "Scissors", "Teddy bear", "Hair drier", "Toothbrush"]

model = yolov5.load('/content/yolov5s.pt')

model.conf = 0.75  # NMS confidence threshold
model.iou = 0.46  # NMS IoU threshold
model.agnostic = False  # NMS class-agnostic
model.multi_label = False  # NMS multiple labels per box
model.max_det = 1000  # maximum number of detections

cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FPS,1)

def draw_boxes(image, boxes, categories, color=(0, 255, 0)):
    for i in range(len(boxes)):
        x1, y1, x2, y2 = map(int, boxes[i])  # Convert tensor values to integers
        cv2.rectangle(image, (x1, y1), (x2, y2), color, 2)
        cv2.putText(image, class_names[int(categories[i])], (x1, y1-2), cv2.FONT_HERSHEY_PLAIN, 0.2, color, 1)
    return image


In [ ]:
model("/content/test.mp4")

In [ ]:

while True:
    ret, frame = cap.read()
    h, w, _ = frame.shape
    results = model(frame)
    predictions = results.pred[0]
    boxes = predictions[:, :4].cpu().numpy()  # Convert tensor to numpy array
    categories = predictions[:, 5].cpu().numpy().astype(int)
    print(categories)
    # Draw bounding boxes on image
    image_with_boxes = draw_boxes(frame, boxes, categories)
    cv2.imshow('frame', image_with_boxes)

    if cv2.waitKey(1) == ord('q'):
        break

cv2.destroyAllWindows()
cap.release()


In [ ]:
input_video_path = '/content/test.mp4'
cap = cv2.VideoCapture(input_video_path)

# Get video properties
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# Define the codec and create VideoWriter object
fourcc = cv2.VideoWriter_fourcc(*'XVID')
output_video_path = '/content/OUt/output_video.avi'
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Process each frame and write to output video
while True:
    ret, frame = cap.read()

    if not ret:
        break

    processed_frame = model(frame)
    out.write(processed_frame)

    cv2.imshow('Processed Frame', processed_frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release everything if job is finished
cap.release()
out.release()
cv2.destroyAllWindows()